In [1]:
import networkx as nx
from sympy import Matrix, pprint, floor
from linkages import *
import openmesh as om

def graph_to_matrix(G):
    M = Matrix()
    for edge in G.edges:
        row = []
        d = edge[0] - edge[1]
        for vertex in G.nodes:
            if vertex == edge[0]  : row.extend(d)
            elif vertex == edge[1]: row.extend(-d)
            else:                   row.extend([0 for _ in range(DIM)])
        M=Matrix([M,row])
    return M

def set_pinning(pins, M: Matrix):
    N = M.nullspace()
    if type(pins) is int: pins = [pins]

    for p in pins:
        for vector in N:
            for i in range(DIM):
                if vector[DIM*(p-1) + i] != 0:
                    vector.fill(0)
                    break
    return N

# helper function to convert nullspace to a list of motions 
def getMotions(N):
    if type(N) is Matrix: N = N.nullspace()
    return [[str(val) + "*v" + str(floor(i/DIM)+1) + ("xyzwrüdiger"[i%DIM]) for i,val in enumerate(vector) if val != 0] for vector in N]

#just a cute function to convert detected motions into a human readible string
def motions_to_string(motions):
    string = ""
    for v in motions:
        if len(v) !=0:
            for val in v:
                if val == v[0] and len(v)>1: word = " depends on "
                elif val != v[len(v)-1]    : word = ", and "
                elif len(v) == 1           : word = " is free\n"
                else                       : word ="\n"
                string += val + word
    return string

def model_to_graph(mesh: om.PolyMesh) -> nx.Graph:
    graph = nx.Graph()

    points = mesh.points()
    for edge in mesh.edge_vertex_indices():
        graph.add_edge(Point(points[edge[0]]), Point(points[edge[1]]))

    return graph    


graph_cool = model_to_graph(om.read_polymesh("models/cube.stl"))

def check_rigidity(M): return M.rank() == M.cols- (DIM+1) * DIM/2
graph = parallel_4_bar(True,True,True)

A = graph_to_matrix(graph_cool)
pprint(A)
print ("the linkage is infinitesimally rigid!" if check_rigidity(A) else "the linkage is infinitesimally flexible")
A = set_pinning(1,A)

print(motions_to_string((getMotions(A))))



⎡-5  0   1   5  0   -1  0  0  0  0   0  0 ⎤
⎢                                         ⎥
⎢0   -5  1   0  0   0   0  0  0  0   5  -1⎥
⎢                                         ⎥
⎢-5  -5  -2  0  0   0   5  5  2  0   0  0 ⎥
⎢                                         ⎥
⎢0   0   0   0  -5  -3  0  5  3  0   0  0 ⎥
⎢                                         ⎥
⎢0   0   0   5  -5  0   0  0  0  -5  5  0 ⎥
⎢                                         ⎥
⎣0   0   0   0  0   0   5  0  3  -5  0  -3⎦
the linkage is infinitesimally rigid!

